In [ ]:
# Using earthaccess access patterns for direct access of cloud-hosted ICESat-2 ATL15 data
# 
# Written 2023-11-14 by Wilson Sauthoff (wsauthoff.github.io)

In [ ]:
# Import libraries
import earthaccess
import matplotlib.pyplot as plt
import os
import xarray as xr

In [ ]:
# Log into NASA Earthdata to search for datasets
earthaccess.login()

In [ ]:
# Find ICESat-2 ATL15 data granules
results = earthaccess.search_data(
    short_name='ATL15',
    version='003',
    cloud_hosted=True,
    bounding_box=(1, -89, -1, -89)  # (lower_left_lon, lower_left_lat , upper_right_lon, upper_right_lat))
)

In [ ]:
# Open data granules as s3 files to stream
files = earthaccess.open(results)
files

In [ ]:
# After viewing files, index the files you wish to open
# Here we will view the highest resolution (1 km) files
print(files[15])
print(files[9])
print(files[8])
print(files[10])

In [ ]:
# Open each file, which are quadrants in polar stereographic coordinations around the Geographic South Pole
ATL15_A1 = xr.open_dataset(files[15], group='delta_h')
ATL15_A2 = xr.open_dataset(files[9], group='delta_h')
ATL15_A3 = xr.open_dataset(files[8], group='delta_h')
ATL15_A4 = xr.open_dataset(files[10], group='delta_h')

In [ ]:
# Specify the variables to keep
variables_to_keep = ['time', 'y', 'x', 'delta_h']

# List of xarray datasets
datasets = [ATL15_A1, ATL15_A2, ATL15_A3, ATL15_A4]

# Function to drop variables not in variables_to_keep from a dataset
def drop_unwanted_variables(dataset):
    variables_to_drop = [var for var in dataset.variables if var not in variables_to_keep]
    return dataset.drop_vars(variables_to_drop)

# Apply the function to each dataset
ATL15_A1, ATL15_A2, ATL15_A3, ATL15_A4 = [drop_unwanted_variables(ds) for ds in datasets]

In [ ]:
# Examine data structure of one of the files
ATL15_A1

In [ ]:
# Plot a visual of the first time slice
ATL15_A1.delta_h[0,:,:].plot()
plt.show()

In [ ]:
# Use xarray concatenation to stitch two quadrants togethers
# Use xarray index selecting to occlude the duplicated x=0 vector of data
ATL15_A12 = xr.concat([ATL15_A2.isel(x=slice(0,-1)), ATL15_A1], dim="x")

In [ ]:
# Plot a visual of the first time slice
ATL15_A12.delta_h[0,:,:].plot()
plt.show()

In [ ]:
# Use xarray concatenation to stitch two quadrants togethers
# Use xarray index selecting to occlude the duplicated x=0 vector of data
ATL15_A34 = xr.concat([ATL15_A3.isel(x=slice(0,-2)), ATL15_A4], dim="x")

In [ ]:
# Plot a visual of the first time slice
ATL15_A34.delta_h[0,:,:].plot()
plt.show()

In [ ]:
# Use xarray concatenation to stitch two quadrants togethers
# Use xarray index selecting to occlude the duplicated y=0 vector of data
ATL15 = xr.concat([ATL15_A34.isel(y=slice(0,-1)), ATL15_A12], dim=("y"))

In [ ]:
# Plot a visual of the first time slice
ATL15.delta_h[0,:,:].plot()
plt.show()

In [ ]:
# Delete variables to reduce memory consumption
del ATL15_A1, ATL15_A12, ATL15_A2, ATL15_A3, ATL15_A34, ATL15_A4